In [1]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, LSTM, Flatten, Reshape, ZeroPadding1D, Conv1D, MaxPooling1D, UpSampling1D, Cropping1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
import gc
import tensorflow as tf
import random
from sklearn import preprocessing
import pickle
from utilities import *

In [2]:
config = loadData("config")
dataSetSize = config["dataSetSize"]
testSetBenignSize = config["testSetBenignSize"]
validationSetSize = config["validationSetSize"]
trainingSetSize = config["trainingSetSize"]
sequenceLen = config["sequenceLen"]
dimensionsCount = config["dimensionsCount"]

### PCA

In [16]:
def trainPCA(codeSize, printSummary = 1, vrbs = 1, return_best = 0, learningRate = 0.001):
    inputSize = sequenceLen * dimensionsCount
    model = Sequential()
    model.add(Dense(inputSize, input_shape=(inputSize,), activation='linear'))
    model.add(Dense(int(codeSize), activation="linear"))
    model.add(Dense(inputSize, activation="linear"))
    if(printSummary == True):
            model.summary()
    adamOptimizer = Adam(learning_rate=learningRate) # , beta_1=0.9, beta_2=0.999, amsgrad=False        
    model.compile(optimizer=adamOptimizer,
        loss='mean_squared_error',
    )
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
    callbacksArray = [es]
    if(return_best):
        mc = ModelCheckpoint('best_pca.h5', monitor='val_loss', mode='min')
        callbacksArray = [es, mc]
    
    trainingSet, testSet, validationSet = getReshapedDataSet(dataSet, "PCA")
    history=model.fit(trainingSet, trainingSet,
                            batch_size=5000,
                            shuffle=True,
                            epochs=10000,                             
                            validation_data=(validationSet, validationSet),
                            callbacks=callbacksArray,
                            verbose = vrbs,
                        )
    
    if(return_best):
        best_model = load_model('best_pca.h5')
        
    returnModel = model
    if(return_best):
        returnModel = best_model
    return [returnModel,min(history.history['val_loss']),len(history.history['val_loss']),history]

In [18]:
codeLayerSize = 50
[model, validatoinLoss, numOfEpochs,_] = trainPCA(codeLayerSize)
model.save('Trained_Model/pca.h5')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 1800)              3241800   
_________________________________________________________________
dense_13 (Dense)             (None, 50)                90050     
_________________________________________________________________
dense_14 (Dense)             (None, 1800)              91800     
Total params: 3,423,650
Trainable params: 3,423,650
Non-trainable params: 0
_________________________________________________________________
Train on 11060 samples, validate on 1382 samples
Epoch 1/10000
11060/11060 [==============================] - 0s 42us/sample - loss: 0.1123 - val_loss: 0.0244
Epoch 2/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 0.0748 - val_loss: 0.0247
Epoch 3/10000
11060/11060 [==============================] - 0s 16us/sample - loss: 0.0691 - val_loss: 0.

11060/11060 [==============================] - 0s 13us/sample - loss: 3.7892e-04 - val_loss: 0.0011
Epoch 139/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 3.6901e-04 - val_loss: 0.0011
Epoch 140/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 3.6312e-04 - val_loss: 0.0011
Epoch 141/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 3.6138e-04 - val_loss: 0.0011
Epoch 142/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 3.5634e-04 - val_loss: 0.0011
Epoch 143/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 3.5906e-04 - val_loss: 0.0011
Epoch 144/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 3.6531e-04 - val_loss: 0.0011
Epoch 145/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 4.0430e-04 - val_loss: 0.0011
Epoch 146/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 4.

11060/11060 [==============================] - 0s 13us/sample - loss: 2.4549e-04 - val_loss: 7.3912e-04
Epoch 276/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 2.5239e-04 - val_loss: 7.3494e-04
Epoch 277/10000
11060/11060 [==============================] - 0s 15us/sample - loss: 2.5280e-04 - val_loss: 7.3363e-04
Epoch 278/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 2.5803e-04 - val_loss: 7.3011e-04
Epoch 279/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 2.5585e-04 - val_loss: 7.2744e-04
Epoch 280/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 2.4416e-04 - val_loss: 7.2430e-04
Epoch 281/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 2.4647e-04 - val_loss: 7.1987e-04
Epoch 282/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 2.5218e-04 - val_loss: 7.2422e-04
Epoch 283/10000
11060/11060 [===========================

11060/11060 [==============================] - 0s 13us/sample - loss: 1.7608e-04 - val_loss: 4.8421e-04
Epoch 412/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 1.7958e-04 - val_loss: 4.8058e-04
Epoch 413/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 1.8789e-04 - val_loss: 4.8155e-04
Epoch 414/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 1.8813e-04 - val_loss: 4.7946e-04
Epoch 415/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 1.9959e-04 - val_loss: 4.8276e-04
Epoch 416/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 1.9597e-04 - val_loss: 4.7461e-04
Epoch 417/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 1.7920e-04 - val_loss: 4.7318e-04
Epoch 418/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 1.7425e-04 - val_loss: 4.7445e-04
Epoch 419/10000
11060/11060 [===========================

11060/11060 [==============================] - 0s 13us/sample - loss: 1.5134e-04 - val_loss: 3.5443e-04
Epoch 548/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 1.3149e-04 - val_loss: 3.5351e-04
Epoch 549/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 1.5654e-04 - val_loss: 3.5514e-04
Epoch 550/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 1.9627e-04 - val_loss: 3.5612e-04
Epoch 551/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 1.8542e-04 - val_loss: 3.5444e-04
Epoch 552/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 1.9175e-04 - val_loss: 3.5328e-04
Epoch 553/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 2.0111e-04 - val_loss: 3.6069e-04
Epoch 554/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 2.4969e-04 - val_loss: 3.5428e-04
Epoch 555/10000
11060/11060 [===========================

### Fully Connected

In [19]:
def trainAutoencoder(numOfHiddenLayersInEncoder, codeLayerSize, printSummary = 1, vrbs = 1, return_best = 1, learningRate = 0.001):
    inputSize = sequenceLen * dimensionsCount
    layerSizeDifference = (inputSize - codeLayerSize) // (numOfHiddenLayersInEncoder + 1)
    model = Sequential()
    
    print(layerSizeDifference)
    
    
    #NeuronsCountInFirstLayer = 64
    model.add(Dense(inputSize, input_shape=(inputSize,), activation='relu'))
    for i in range(1,numOfHiddenLayersInEncoder + 1):
        model.add(Dense(int(inputSize - (layerSizeDifference * i)), activation="relu"))
        
    model.add(Dense(int(codeLayerSize), activation="relu"))
        
    for j in range(1,numOfHiddenLayersInEncoder + 1):    
        model.add(Dense(int(inputSize - (layerSizeDifference * (numOfHiddenLayersInEncoder - j + 1))), activation="relu"))
        
    model.add(Dense(inputSize, activation="linear"))

    if(printSummary == True):
            model.summary()

    adamOptimizer = Adam(learning_rate=learningRate) # , beta_1=0.9, beta_2=0.999, amsgrad=False        
            
    model.compile(optimizer=adamOptimizer,
        loss='mean_squared_error',
    )
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
    callbacksArray = [es]
    if(return_best):
        mc = ModelCheckpoint('best_Autoencoder.h5', monitor='val_loss', mode='min')
        callbacksArray = [es, mc]

    trainingSet, testSet, validationSet = getReshapedDataSet(dataSet, "fullyConnected")
    history=model.fit(trainingSet, trainingSet,
                            batch_size=5000,
                            shuffle=True,
                            epochs=10000,                             
                            validation_data=(validationSet, validationSet),
                            callbacks=callbacksArray,
                            verbose = vrbs,
                        )
    
    if(return_best):
        best_model = load_model('best_Autoencoder.h5')
        
    returnModel = model
    if(return_best):
        returnModel = best_model
    return [returnModel,min(history.history['val_loss']),len(history.history['val_loss']),history]

In [20]:
numOfHiddenLayers = 1
codeLayerSize = 40 # 64
[model, validatoinLoss, numOfEpochs, history] = trainAutoencoder(numOfHiddenLayers, codeLayerSize, return_best = 1)
model.save('Trained_Model/autoencoder.h5')

880
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 1800)              3241800   
_________________________________________________________________
dense_16 (Dense)             (None, 920)               1656920   
_________________________________________________________________
dense_17 (Dense)             (None, 40)                36840     
_________________________________________________________________
dense_18 (Dense)             (None, 920)               37720     
_________________________________________________________________
dense_19 (Dense)             (None, 1800)              1657800   
Total params: 6,631,080
Trainable params: 6,631,080
Non-trainable params: 0
_________________________________________________________________
Train on 11060 samples, validate on 1382 samples
Epoch 1/10000
11060/11060 [==============================

Epoch 63/10000
11060/11060 [==============================] - 0s 35us/sample - loss: 2.4462e-04 - val_loss: 6.5521e-04
Epoch 64/10000
11060/11060 [==============================] - 0s 36us/sample - loss: 2.4213e-04 - val_loss: 6.5098e-04
Epoch 65/10000
11060/11060 [==============================] - 0s 37us/sample - loss: 2.3960e-04 - val_loss: 6.4204e-04
Epoch 66/10000
11060/11060 [==============================] - 0s 35us/sample - loss: 2.3774e-04 - val_loss: 6.3700e-04
Epoch 67/10000
11060/11060 [==============================] - 0s 38us/sample - loss: 2.3661e-04 - val_loss: 6.3123e-04
Epoch 68/10000
11060/11060 [==============================] - 0s 36us/sample - loss: 2.3527e-04 - val_loss: 6.2643e-04
Epoch 69/10000
11060/11060 [==============================] - 0s 40us/sample - loss: 2.3501e-04 - val_loss: 6.2325e-04
Epoch 70/10000
11060/11060 [==============================] - 0s 38us/sample - loss: 2.4572e-04 - val_loss: 7.0155e-04
Epoch 71/10000
11060/11060 [====================

### 1D Conv

In [6]:
def train1DConv(numOfHiddenLayersInEncoder, FiltersCountInFirstLayer = 32, printSummary = 1, vrbs = 1, return_best = 1, filterSize = 3, learningRate = 0.001):
    numOfHiddenLayersInEncoder = numOfHiddenLayersInEncoder + 1
    model = Sequential()    
    poolingSize = 2
    numOfFiltersInEncoder = [FiltersCountInFirstLayer]
    paddingSize = (filterSize // 2)
    inputLen = (sequenceLen * dimensionsCount)
    paddedInputLength = inputLen + (paddingSize*2)
    encoderLayersFilterSizes = [paddedInputLength]
    model.add(ZeroPadding1D(paddingSize,input_shape=(sequenceLen * dimensionsCount,1)))
    model.add(Conv1D(int(FiltersCountInFirstLayer), filterSize, activation='relu', padding = 'same'))
    for i in range(1,numOfHiddenLayersInEncoder):
        model.add(MaxPooling1D(poolingSize, padding='same'))
        model.add(Conv1D(int(FiltersCountInFirstLayer*np.power(2,i)), filterSize, activation='relu', padding = 'same'))
        
        encoderLayersFilterSizes.append(int(np.ceil(paddedInputLength/np.power(2,i))))
        numOfFiltersInEncoder.append(int(FiltersCountInFirstLayer*np.power(2,i)))
    model.add(Flatten())
    model.add(Reshape((encoderLayersFilterSizes[-1], numOfFiltersInEncoder[-1])))
    for j in range(1,numOfHiddenLayersInEncoder):    
        model.add(Conv1D(numOfFiltersInEncoder[-j], filterSize, activation='relu', padding = 'same'))
        model.add(UpSampling1D(poolingSize))

    model.add(Conv1D(numOfFiltersInEncoder[0], filterSize, activation='relu', padding='same'))
    model.add(Conv1D(1, filterSize, activation='linear', padding='same'))
    
    toCrop = (model.layers[-1].output_shape[1] - inputLen) // 2
    
    model.add(Cropping1D(toCrop))
    model.summary()

    adamOptimizer = Adam(learning_rate=learningRate) # , beta_1=0.9, beta_2=0.999, amsgrad=False            
    model.compile(optimizer=adamOptimizer,
        loss='mean_squared_error',
    )

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
    callbacksArray = [es]
    if(return_best):
        mc = ModelCheckpoint('best_1DConv.h5', monitor='val_loss', mode='min')
        callbacksArray = [es, mc]

    trainingSet, testSet, validationSet = getReshapedDataSet(dataSet, "1DConv")
    history=model.fit(trainingSet, trainingSet,
                            batch_size=1000,
                            shuffle=True,
                            epochs=10000,                             
                            validation_data=(validationSet, validationSet),
                            callbacks=callbacksArray,
                            verbose = vrbs,
                        )
    
    if(return_best):
        best_model = load_model('best_1DConv.h5')
        
    returnModel = model
    if(return_best):
        returnModel = best_model
    return [returnModel,min(history.history['val_loss']),len(history.history['val_loss']),history]

In [7]:
numOfLayers = 2
filtersCountInFirstLayer = 48
[model, validatoinLoss, numOfEpochs, history] = train1DConv(numOfLayers, filtersCountInFirstLayer, filterSize = 4)
model.save('Trained_Model/conv.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding1d_1 (ZeroPaddin (None, 1804, 1)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1804, 48)          240       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 902, 48)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 902, 96)           18528     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 451, 96)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 451, 192)          73920     
_________________________________________________________________
flatten_1 (Flatten)          (None, 86592)            

11060/11060 [==============================] - 4s 351us/sample - loss: 2.8359e-05 - val_loss: 4.9239e-05
Epoch 119/10000
11060/11060 [==============================] - 4s 349us/sample - loss: 1.6863e-05 - val_loss: 4.7531e-05
Epoch 120/10000
11060/11060 [==============================] - 4s 346us/sample - loss: 1.1806e-05 - val_loss: 4.6918e-05
Epoch 121/10000
11060/11060 [==============================] - 4s 346us/sample - loss: 1.2905e-05 - val_loss: 4.9627e-05
Epoch 122/10000
11060/11060 [==============================] - 4s 353us/sample - loss: 4.9530e-05 - val_loss: 5.8137e-05
Epoch 123/10000
11060/11060 [==============================] - 4s 350us/sample - loss: 3.6674e-05 - val_loss: 4.7653e-05
Epoch 124/10000
11060/11060 [==============================] - 4s 347us/sample - loss: 1.7811e-05 - val_loss: 4.6687e-05
Epoch 125/10000
11060/11060 [==============================] - 4s 363us/sample - loss: 1.3179e-05 - val_loss: 4.8415e-05
Epoch 126/10000
11060/11060 [===================

Epoch 186/10000
11060/11060 [==============================] - 4s 352us/sample - loss: 9.2573e-06 - val_loss: 3.1722e-05
Epoch 187/10000
11060/11060 [==============================] - 4s 351us/sample - loss: 1.4101e-05 - val_loss: 3.8340e-05
Epoch 188/10000
11060/11060 [==============================] - 4s 353us/sample - loss: 3.2987e-05 - val_loss: 3.2915e-05
Epoch 189/10000
11060/11060 [==============================] - 4s 347us/sample - loss: 1.1767e-05 - val_loss: 3.1113e-05
Epoch 190/10000
11060/11060 [==============================] - 4s 358us/sample - loss: 8.2761e-06 - val_loss: 3.0965e-05
Epoch 191/10000
11060/11060 [==============================] - 4s 351us/sample - loss: 8.0128e-06 - val_loss: 3.2318e-05
Epoch 192/10000
11060/11060 [==============================] - 4s 352us/sample - loss: 3.2639e-05 - val_loss: 6.7828e-05
Epoch 193/10000
11060/11060 [==============================] - 4s 353us/sample - loss: 6.2430e-05 - val_loss: 3.4922e-05
Epoch 194/10000
11060/11060 [===

### LSTM 

In [13]:
def trainLSTM(numOfLayers, numOfNeurons, printSummary = 1, vrbs = 1, return_best = 1, learningRate = 0.001):
        sequenceLen = dataSet.shape[1]
        dimensionsCount = dataSet.shape[2]
        
        print('Training model.')
        model = Sequential()
        model.add(LSTM(numOfNeurons, input_shape=(sequenceLen, dimensionsCount), return_sequences=True))
        for j in range(0, numOfLayers - 1):
            model.add(LSTM(numOfNeurons, return_sequences=True))

        model.add(Dense(dimensionsCount))
        model.add(Activation("linear"))

        if(printSummary == True):
            model.summary()

        adamOptimizer = Adam(learning_rate=learningRate) # , beta_1=0.9, beta_2=0.999, amsgrad=False            
        model.compile(optimizer=adamOptimizer,
            loss='mean_squared_error',
        )

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
        callbacksArray = [es]
        if(return_best):
            mc = ModelCheckpoint('best_lstm.h5', monitor='val_loss', mode='min')
            callbacksArray = [es, mc]

        trainingSet, testSet, validationSet = getReshapedDataSet(dataSet, "LSTM")
        history=model.fit(trainingSet, trainingSet,
                            batch_size=5000,
                            shuffle=True,
                            epochs=10000,                             
                            validation_data=(validationSet, validationSet),
                            callbacks=callbacksArray,
                            verbose = vrbs,
                        )

        if(return_best):
            best_model = load_model('best_lstm.h5')
        returnModel = model
        if(return_best):
            returnModel = best_model
        return [returnModel,min(history.history['val_loss']),len(history.history['val_loss']),history]

In [14]:
numOfLayers = 1
numOfNeurons = 20 
[model, validatoinLoss, numOfEpochs, history] = trainLSTM(numOfLayers, numOfNeurons)
model.save('Trained_Model/lstm.h5')

Training model.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 20)           2400      
_________________________________________________________________
dense (Dense)                (None, 200, 9)            189       
_________________________________________________________________
activation (Activation)      (None, 200, 9)            0         
Total params: 2,589
Trainable params: 2,589
Non-trainable params: 0
_________________________________________________________________
Train on 11060 samples, validate on 1382 samples
Epoch 1/10000
11060/11060 [==============================] - 2s 138us/sample - loss: 0.1067 - val_loss: 0.0294
Epoch 2/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 0.0954 - val_loss: 0.0283
Epoch 3/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 0.0859 - val

Epoch 140/10000
11060/11060 [==============================] - 0s 40us/sample - loss: 8.6888e-04 - val_loss: 0.0025
Epoch 141/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 8.6277e-04 - val_loss: 0.0025
Epoch 142/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 8.5672e-04 - val_loss: 0.0024
Epoch 143/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 8.5077e-04 - val_loss: 0.0024
Epoch 144/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 8.4488e-04 - val_loss: 0.0024
Epoch 145/10000
11060/11060 [==============================] - 0s 21us/sample - loss: 8.3905e-04 - val_loss: 0.0024
Epoch 146/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 8.3328e-04 - val_loss: 0.0023
Epoch 147/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 8.2758e-04 - val_loss: 0.0023
Epoch 148/10000
11060/11060 [==============================] - 0s 18us/s

Epoch 279/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 4.0585e-04 - val_loss: 6.5438e-04
Epoch 280/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 4.0413e-04 - val_loss: 6.5024e-04
Epoch 281/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 4.0242e-04 - val_loss: 6.4649e-04
Epoch 282/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 4.0072e-04 - val_loss: 6.4300e-04
Epoch 283/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 3.9904e-04 - val_loss: 6.3954e-04
Epoch 284/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 3.9737e-04 - val_loss: 6.3590e-04
Epoch 285/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 3.9572e-04 - val_loss: 6.3189e-04
Epoch 286/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 3.9407e-04 - val_loss: 6.2815e-04
Epoch 287/10000
11060/11060 [===========

11060/11060 [==============================] - 0s 18us/sample - loss: 2.5156e-04 - val_loss: 3.9493e-04
Epoch 416/10000
11060/11060 [==============================] - 0s 34us/sample - loss: 2.5080e-04 - val_loss: 3.9397e-04
Epoch 417/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 2.5004e-04 - val_loss: 3.9297e-04
Epoch 418/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 2.4929e-04 - val_loss: 3.9198e-04
Epoch 419/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 2.4854e-04 - val_loss: 3.9104e-04
Epoch 420/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 2.4779e-04 - val_loss: 3.9017e-04
Epoch 421/10000
11060/11060 [==============================] - 0s 21us/sample - loss: 2.4705e-04 - val_loss: 3.8936e-04
Epoch 422/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 2.4631e-04 - val_loss: 3.8854e-04
Epoch 423/10000
11060/11060 [===========================

11060/11060 [==============================] - 0s 27us/sample - loss: 1.7151e-04 - val_loss: 3.0473e-04
Epoch 552/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.7106e-04 - val_loss: 3.0430e-04
Epoch 553/10000
11060/11060 [==============================] - 0s 35us/sample - loss: 1.7061e-04 - val_loss: 3.0386e-04
Epoch 554/10000
11060/11060 [==============================] - 0s 29us/sample - loss: 1.7016e-04 - val_loss: 3.0334e-04
Epoch 555/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.6970e-04 - val_loss: 3.0291e-04
Epoch 556/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.6926e-04 - val_loss: 3.0246e-04
Epoch 557/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.6881e-04 - val_loss: 3.0198e-04
Epoch 558/10000
11060/11060 [==============================] - 0s 26us/sample - loss: 1.6836e-04 - val_loss: 3.0145e-04
Epoch 559/10000
11060/11060 [===========================

...

Epoch 2850/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 2.0579e-06 - val_loss: 1.1734e-05
Epoch 2851/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 2.0724e-06 - val_loss: 1.1731e-05
Epoch 2852/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 2.0742e-06 - val_loss: 1.1705e-05
Epoch 2853/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 2.0598e-06 - val_loss: 1.1692e-05
Epoch 2854/10000
11060/11060 [==============================] - 0s 26us/sample - loss: 2.0503e-06 - val_loss: 1.1675e-05
Epoch 2855/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 2.0458e-06 - val_loss: 1.1661e-05
Epoch 2856/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 2.0416e-06 - val_loss: 1.1647e-05
Epoch 2857/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 2.0373e-06 - val_loss: 1.1628e-05
Epoch 2858/10000
11060/11060 [==

Epoch 2985/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 1.7865e-06 - val_loss: 9.9646e-06
Epoch 2986/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 1.7760e-06 - val_loss: 9.9461e-06
Epoch 2987/10000
11060/11060 [==============================] - 0s 28us/sample - loss: 1.7627e-06 - val_loss: 9.9349e-06
Epoch 2988/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.7576e-06 - val_loss: 9.9192e-06
Epoch 2989/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.7544e-06 - val_loss: 9.9120e-06
Epoch 2990/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.7555e-06 - val_loss: 9.8972e-06
Epoch 2991/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 1.7581e-06 - val_loss: 9.8862e-06
Epoch 2992/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 1.7578e-06 - val_loss: 9.8775e-06
Epoch 2993/10000
11060/11060 [==

Epoch 3120/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 1.5284e-06 - val_loss: 8.5265e-06
Epoch 3121/10000
11060/11060 [==============================] - 0s 21us/sample - loss: 1.5232e-06 - val_loss: 8.5168e-06
Epoch 3122/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 1.5246e-06 - val_loss: 8.5068e-06
Epoch 3123/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 1.5302e-06 - val_loss: 8.5101e-06
Epoch 3124/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 1.5368e-06 - val_loss: 8.4888e-06
Epoch 3125/10000
11060/11060 [==============================] - 0s 16us/sample - loss: 1.5527e-06 - val_loss: 8.4974e-06
Epoch 3126/10000
11060/11060 [==============================] - 0s 21us/sample - loss: 1.5929e-06 - val_loss: 8.4877e-06
Epoch 3127/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.6643e-06 - val_loss: 8.5554e-06
Epoch 3128/10000
11060/11060 [==

Epoch 3255/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.3416e-06 - val_loss: 7.3725e-06
Epoch 3256/10000
11060/11060 [==============================] - 0s 16us/sample - loss: 1.3272e-06 - val_loss: 7.3640e-06
Epoch 3257/10000
11060/11060 [==============================] - 0s 22us/sample - loss: 1.3306e-06 - val_loss: 7.3527e-06
Epoch 3258/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 1.3257e-06 - val_loss: 7.3469e-06
Epoch 3259/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.3268e-06 - val_loss: 7.3401e-06
Epoch 3260/10000
11060/11060 [==============================] - 0s 19us/sample - loss: 1.3279e-06 - val_loss: 7.3332e-06
Epoch 3261/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 1.3255e-06 - val_loss: 7.3215e-06
Epoch 3262/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.3285e-06 - val_loss: 7.3247e-06
Epoch 3263/10000
11060/11060 [==

Epoch 3390/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.2769e-06 - val_loss: 6.5077e-06
Epoch 3391/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.4750e-06 - val_loss: 6.5873e-06
Epoch 3392/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 1.7243e-06 - val_loss: 6.5973e-06
Epoch 3393/10000
11060/11060 [==============================] - 0s 24us/sample - loss: 1.7671e-06 - val_loss: 6.5137e-06
Epoch 3394/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.5453e-06 - val_loss: 6.4846e-06
Epoch 3395/10000
11060/11060 [==============================] - 0s 17us/sample - loss: 1.3612e-06 - val_loss: 6.4582e-06
Epoch 3396/10000
11060/11060 [==============================] - 0s 18us/sample - loss: 1.3588e-06 - val_loss: 6.4543e-06
Epoch 3397/10000
11060/11060 [==============================] - 0s 29us/sample - loss: 1.4434e-06 - val_loss: 6.4503e-06
Epoch 03397: early stopping
